In [156]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [157]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.atl_events_db
collection = db.events

In [158]:
#set up chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.15/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\steve\.wdm\drivers\chromedriver\win32\93.0.4577.15]


In [159]:
#visit url
url = "https://terminalwestatl.com/calendar"
request = requests.get(url)
browser.visit(url)

In [160]:
#wait for browser to load calendar into html
time.sleep(5)

In [161]:
#parse HTML
html = browser.html
soup = bs(html, "html.parser")
browser.quit()

In [162]:
#make lists for dataframe
event_name = []
event_desc = []
event_date = []
event_time = []
event_link = []

In [163]:
#make list of all the event cards
results = soup.find_all('div', class_='c-axs-event-card__wrapper')

In [164]:
#loop through event cards
for result in results:

    #find Headlining artist or event title
    title = result.find('div', class_='c-axs-event-card__title c-axs-event-card__title7124').text
    #find opening artists and make an event description
    try:
        supporting = result.find('div', class_="c-axs-event-card__supporting-text c-axs-event-card__supporting-text7124").text
        if len(supporting) > 0:
            description = f'{title} performing with {supporting}'
    except:
        supporting = ""
        description = "n/a"
    #find date/time, then split to make a column for each
    try:
        date_time = result.find('p', class_='c-axs-event-card__date--showtime7124').text

        date = date_time.split("2021")
        time = date_time.split("2021")[1]
    except:
        date = "TBD"
        time = 'TBD'
    #find link
    url = result.find('a', class_='c-axs-event-card__header c-axs-event-card__header7124')['href']
    link = f'https://terminalwestatl.com{url}'

    event_name.append(title)
    event_desc.append(description)
    event_date.append(f'{date[0]} 2021')
    event_time.append(f'{date[1]}')
    event_link.append(link)

    event = {
        "Event_name":title,
        "Event_description":description,
        "Event_date":f'{date[0]} 2021',
        "Event_time":f'{date[1]}',
        "Event_link":link
    }
    
    collection.insert_one(event)


In [165]:
#make dataframe
df = pd.DataFrame({
    "Event_name":event_name,
    "Event_description":event_desc,
    "Event_date":event_date,
    "Event_time":event_time,
    "Event_link":event_link
})

In [166]:
df.to_csv("terminal_west.csv")